In [1]:
import sys
from pathlib import Path

# Add the parent directory to the Python path
sys.path.append(str(Path().resolve().parent))

from spytial import diagram
from spytial.annotations import orientation, attribute, hideAtom, atomColor, group, flag
from spytial import relationalizer, RelationalizerBase, Atom, Relation

import dd
print(dd.__version__)

0.6.0


## DD is a standard Python lib for BDDs

- But no clear obv visualizer beyond dumping to network X and dot graphs.
- https://github.com/tulip-control/dd?tab=readme-ov-file



In [2]:
from dd.autoref import BDD

bdd = BDD()
bdd.declare('x', 'y', 'z')
u = bdd.add_expr(r'(x /\ y) \/ ~z')

# Map variable indices to names for readability
var_names = {idx: name for name, idx in bdd.vars.items()}

print("BDD node structure (node_id: var_name, low ->, high ->):")
for node, (var, low, high) in bdd._bdd._succ.items():
    var_label = var_names.get(var, str(var))
    print(f"Node {node}: var={var_label}, low={low}, high={high}")

print(f"\nBDD terminal info:")
print(f"False terminal: {bdd.false.node}")
print(f"True terminal: {bdd.true.node}")
print(f"Root node u: {u.node}")

print(f"\nAll referenced node IDs in relations:")
all_refs = set()
for node, (var, low, high) in bdd._bdd._succ.items():
    if low is not None:
        all_refs.add(low)
    if high is not None:
        all_refs.add(high)
print(f"Referenced IDs: {sorted(all_refs)}")

print(f"\nNodes that exist in _succ: {sorted(bdd._bdd._succ.keys())}")

#bdd.dump('bdd.dot')

# # Dump to JSON
# filename = 'bdd.json'
# roots = dict(u=u)
# bdd.dump(filename, roots)

BDD node structure (node_id: var_name, low ->, high ->):
Node 1: var=3, low=None, high=None
Node 2: var=x, low=-1, high=1
Node 3: var=y, low=-1, high=1
Node 4: var=x, low=-1, high=3
Node 5: var=z, low=-1, high=1
Node 6: var=y, low=-5, high=1
Node 7: var=x, low=-5, high=6

BDD terminal info:
False terminal: -1
True terminal: 1
Root node u: 7

All referenced node IDs in relations:
Referenced IDs: [-5, -1, 1, 3, 6]

Nodes that exist in _succ: [1, 2, 3, 4, 5, 6, 7]


In [4]:
@relationalizer(priority=101)
class BDDRelationalizer(RelationalizerBase):
    def can_handle(self, obj):
        return isinstance(obj, BDD)

    def relationalize(self, obj, walker_func):
        atoms = []
        relations = []
        
        # Build variable index to name mapping from BDD object
        var_names = {idx: name for name, idx in obj.vars.items()}
        seen = set()
        
        # Get terminal node IDs from dd library constants
        false_id = obj.false.node  # Usually -1
        true_id = obj.true.node    # Usually 1
        
        # Add terminal atoms first
        atoms.append(Atom(id=str(true_id), type='Terminal', label='True'))
        atoms.append(Atom(id=str(false_id), type='Terminal', label='False'))
        seen.add(true_id)
        seen.add(false_id)
        
        # Add variable level atoms and relations
        level_atoms = {}
        for var_name, level in obj.vars.items():
            level_id = f"level_{level}"
            if level_id not in seen:
                atoms.append(Atom(id=level_id, type='Level', label=f'Level {level}'))
                level_atoms[level] = level_id
                seen.add(level_id)
        
        # Process all decision nodes from BDD structure
        for node_id, (var_idx, low_child, high_child) in obj._bdd._succ.items():
            # Skip if this is a terminal node (already added above)
            if node_id in (true_id, false_id):
                continue
                
            # Check if this is a terminal node by having no children
            if low_child is None and high_child is None:
                # This might be an additional terminal - skip if already handled
                if node_id not in seen:
                    atoms.append(Atom(id=str(node_id), type='Terminal', label=f'Terminal_{node_id}'))
                    seen.add(node_id)
                continue
            
            # This is a decision node
            if var_idx in var_names:
                var_name = var_names[var_idx]
            else:
                var_name = f"var_{var_idx}"
                
            atoms.append(Atom(id=str(node_id), type='BDDNode', label=var_name))
            seen.add(node_id)
            
            # Add level relation: this node is at this level
            if var_idx in level_atoms:
                level_id = level_atoms[var_idx]
                relations.append(Relation(name='at_level', atoms=[str(node_id), level_id]))
            
            # Add relations for children with proper edge types
            if low_child is not None:
                target_id, is_complemented = self._resolve_target(low_child, true_id, false_id)
                edge_type = 'low_complemented' if is_complemented else 'low'
                relations.append(Relation(name=edge_type, atoms=[str(node_id), str(target_id)]))
                
            if high_child is not None:
                target_id, is_complemented = self._resolve_target(high_child, true_id, false_id)
                edge_type = 'high_complemented' if is_complemented else 'high'
                relations.append(Relation(name=edge_type, atoms=[str(node_id), str(target_id)]))
        
        # Add level ordering relations (level 0 < level 1 < level 2, etc.)
        sorted_levels = sorted(obj.vars.values())
        for i in range(len(sorted_levels) - 1):
            current_level = f"level_{sorted_levels[i]}"
            next_level = f"level_{sorted_levels[i + 1]}"
            relations.append(Relation(name='precedes', atoms=[current_level, next_level]))
        
        return atoms, relations
    
    def _resolve_target(self, child_id, true_id, false_id):
        """
        Resolve a child ID to its positive form and determine if it's complemented.
        Returns: (positive_id, is_complemented)
        """
        if isinstance(child_id, str):
            # Handle serialized forms
            if child_id == "T":
                return ("T", False)
            elif child_id == "F":
                return ("F", False)
            else:
                return (child_id, False)
        
        # Handle integer IDs
        if child_id < 0:
            # Negative ID means complemented edge to positive node
            pos_id = abs(child_id)
            return (pos_id, True)
        else:
            # Positive ID means regular edge
            return (child_id, False)

# Test the relationalizer
print("Testing BDD relationalizer with level relations:")
atoms, relations = BDDRelationalizer().relationalize(bdd, None)
print(f"Generated {len(atoms)} atoms and {len(relations)} relations")

# Show atoms with their labels
print("\nAtoms (including level nodes):")
for atom in sorted(atoms, key=lambda x: int(x.id) if x.id.lstrip('-').isdigit() else 999):
    print(f"  {atom.id}: {atom.label} ({atom.type})")

# Show relations with edge types
print(f"\nRelations (including level relations):")
for rel in relations:
    if rel.name == 'at_level':
        print(f"  {rel.name}: {rel.atoms[0]} at {rel.atoms[1]}")
    elif rel.name == 'precedes':
        print(f"  {rel.name}: {rel.atoms[0]} precedes {rel.atoms[1]}")
    else:
        edge_style = "dashed" if "complemented" in rel.name else "solid"
        edge_label = rel.name.replace("_complemented", "")
        print(f"  {edge_label} ({edge_style}): {rel.atoms[0]} -> {rel.atoms[1]}")

diagram(bdd)


Testing BDD relationalizer with level relations:
Generated 11 atoms and 20 relations

Atoms (including level nodes):
  -1: False (Terminal)
  1: True (Terminal)
  2: x (BDDNode)
  3: y (BDDNode)
  4: x (BDDNode)
  5: z (BDDNode)
  6: y (BDDNode)
  7: x (BDDNode)
  level_0: Level 0 (Level)
  level_1: Level 1 (Level)
  level_2: Level 2 (Level)

Relations (including level relations):
  at_level: 2 at level_0
  low (dashed): 2 -> 1
  high (solid): 2 -> 1
  at_level: 3 at level_1
  low (dashed): 3 -> 1
  high (solid): 3 -> 1
  at_level: 4 at level_0
  low (dashed): 4 -> 1
  high (solid): 4 -> 3
  at_level: 5 at level_2
  low (dashed): 5 -> 1
  high (solid): 5 -> 1
  at_level: 6 at level_1
  low (dashed): 6 -> 5
  high (solid): 6 -> 1
  at_level: 7 at level_0
  low (dashed): 7 -> 5
  high (solid): 7 -> 6
  precedes: level_0 precedes level_1
  precedes: level_1 precedes level_2
